# Creating the containers for .NET notebooks for exploring dbatools

To run these notebooks, you will first have to have set up Jupyter Notebooks following the information here https://sqldbawithabeard.com/2020/02/07/new-net-notebooks-are-here-powershell-7-notebooks-are-here/

![dbatools](.\images\dbatools.png )

We have written a book which will give you a brilliant introduction to dbatools. It's called dbatools in a Month of Lunches and you can find it at https://beard.media/book

# Setting up the containers for the rest of the dbachecks notebooks

To be able to follow along with the rest of the notebooks, you will need to set up two containers. 

This notebook will enable you to do that.

You will need to have Docker installed. The image is based on the SQL Server 2019 image so you will need to have docker set to use Linux Containers. Right click on the Docker icon in the notification area and if it says "Switch to Linux Containers" click to switch and wait for Docker to restart.  

![Switch To Linux Containers](.\images\switchtolinuxcontainers.png )

You will be able to run all of the code in the notebooks by creating the folder, credential and containers in this notebook and then you can click on the play button in each code block to run the code. Note - There are a few code blocks with the results already included which should not be run. They are to show you the results of a command that cannot be run against containers (setting up configuration for domain accounts for example)

**You can also navigate through the Notebook using the arrow keys and press SHIFT + ENTER to run a cell**

## Create the folders and the credential

The code below will create a directory called dbatools-demo in your Documents folder and save a credential file for logging into the containers. You can alter the directory created by altering the `$FolderPath`  

The code has been collapsed but you can view it using the view menu in the lab or by clicking the 3 ellipses

In [10]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'

########################################################
Write-Output "Creating Directory $FolderPath"
if(Test-Path $FolderPath){
    Write-Output "Path $FolderPath exists already"
}else {
 $null =    New-Item $FolderPath -ItemType Directory
}

Write-Output "Creating Directory $FolderPath\SQL1"
if(Test-Path "$FolderPath\SQL1"){
    Write-Output "Directory SQL1 exists already"
    Get-ChildItem "$FolderPath\SQL1" -Recurse| Remove-Item -Recurse -Force
}else {
$null = New-Item "$FolderPath\SQL1"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL1\dummyfile.txt"
if(Test-Path "$FolderPath\SQL1\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL1\dummyfile.txt" -ItemType file
}

Write-Output "Creating Directory $FolderPath\SQL2"
if(Test-Path "$FolderPath\SQL2"){
    Write-Output "Directory SQL2 exists already"
    Get-ChildItem "$FolderPath\SQL2" -Recurse| Remove-Item -Recurse -Force
}else {
 $null =    New-Item "$FolderPath\SQL2"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL2\dummyfile.txt"
if(Test-Path "$FolderPath\SQL2\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL2\dummyfile.txt" -ItemType file
}

Write-Output "Creating a credential file for the containers - Please don't do this in production"

$sqladminPassword = ConvertTo-SecureString 'dbatools.IO' -AsPlainText -Force 
$cred = New-Object System.Management.Automation.PSCredential ('sqladmin', $sqladminpassword)
$Cred | Export-Clixml -Path $FolderPath\sqladmin.cred
Write-Output "Credential file created"

Write-Output "Setting the docker-compose files values"
$dockercompose = (Get-Content .\dockercompose.yml) -replace '__ReplaceME__' , $FolderPath
# $dockercompose
$dockercompose | Set-Content .\docker-compose.yml



Creating Directory C:\Users\mrrob\Documents\dbatoolsdemo
Creating Directory C:\Users\mrrob\Documents\dbatoolsdemo\SQL1
Creating File C:\Users\mrrob\Documents\dbatoolsdemo\SQL1\dummyfile.txt
Creating Directory C:\Users\mrrob\Documents\dbatoolsdemo\SQL2
Creating File C:\Users\mrrob\Documents\dbatoolsdemo\SQL2\dummyfile.txt
Creating a credential file for the containers - Please don't do this in production
Credential file created
Setting the docker-compose files values


## Containers

We are going to create two SQL 2019 containers using the sqldbawithabeard\dbachecks [image from the Docker Hub](https://hub.docker.com/repository/docker/sqldbawithabeard/dbachecksdemo). 

The first time it is going to pull the image sqldbawithabeard/dbachecksdemo from the Docker Hub. If you wish to do this first, you can run the cell below



In [15]:
docker pull sqldbawithabeard/dbachecksdemo

Using default tag: latest
latest: Pulling from sqldbawithabeard/dbachecksdemo
b234f539f7a1: Already exists
55172d420b43: Already exists
5ba5bbeb6b91: Already exists
43ae2841ad7a: Already exists
f6c9c6de4190: Already exists
fd449e8d7345: Already exists
51d0933375e5: Already exists
64f21ba81504: Already exists
55b6919c0cc6: Already exists
76386ba2eb1b: Pulling fs layer
76386ba2eb1b: Verifying Checksum
76386ba2eb1b: Download complete
76386ba2eb1b: Pull complete
Digest: sha256:6332521d6426eff4a6ed148545d34d28e9f979380400319b54fc98cff48936bd
Status: Downloaded newer image for sqldbawithabeard/dbachecksdemo:latest
docker.io/sqldbawithabeard/dbachecksdemo:latest


Now we can start the containers with the code below

In [11]:
docker-compose up -d

Creating network "dbatools_BeardsMagicNetwork" with the default driver
Creating dbatools_SQL2019-1_1 ... 

Creating dbatools_SQL2019_1   ... 


Creating dbatools_SQL2019-1_1 ... done

Creating dbatools_SQL2019_1   ... done



All being well, you wil have something that looks like


![DockerCompose](.\images\containers.png )

If you get an error you might need to add the user you have shared your drives with Docker modify permissions to the \Documents\ directory in your user profile   
More details https://docs.docker.com/docker-for-windows/ or troubleshoot in the normal way


Now we can start exploring with dbatools :-)

If you have not installed dbatools, it can be got from the PowerShell Gallery using `Install-Module dbatools` the code below will check for the module and either install it in your user profile or update it and Import it

In [ ]:
if(Get-Module dbatools -ListAvailable){
Write-Output "Updating dbatools"
Update-Module dbatools
}else {
Write-Output "Installing dbatools in your user profile"
Install-Module dbatools -Scope CurrentUser
}
Import-Module dbatools

## Check connection
Now that is done, we can make a connection to our instances and see if we can connect to them.

We are going to use `Connect-DBaInstance` to do this and we will use the containers that we have created and because we need to use SQL Authentication, we will use a credential that we have saved to disk using `Export-CliXML` in one of the celss above.

It is **Important** to state that this is not a production secure solution and should not be used as a means of accessing any secure system.

In [5]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}

 Creating connection to the containers
We have a connection to the containers


If there are warnings above - Check the containers are running - Look at the status column

In [6]:
docker ps -a

CONTAINER ID IMAGE COMMAND CREATED STATUS PORTS NAMES

e5e6a324ee39 sqldbawithabeard/dbachecksdemo "/opt/mssql/bin/perm…" 42 seconds ago Up 40 seconds 0.0.0.0:15593->1433/tcp dbatools_SQL2019-1_1

01a539677f71 sqldbawithabeard/dbachecksdemo "/opt/mssql/bin/perm…" 42 seconds ago Up 35 seconds 0.0.0.0:15592->1433/tcp dbatools_SQL2019_1

If there are warnings above and the containers are running - check the logs

In [7]:
docker logs dbatools_SQL2019_1

SQL Server 2019 will run as non-root by default.

This container is running as user mssql.

Your master database file is owned by mssql.

To learn more visit https://go.microsoft.com/fwlink/?linkid=2099216.

2020-03-05 09:02:50.10 Server Microsoft SQL Server 2019 (RTM) - 15.0.2000.5 (X64)

...

2020-03-05 09:03:20.42 spid42s InitializeXdbPkgLauncher failed. ErrorCode: 0x80004005.

2020-03-05 09:03:20.42 spid42s SQL Server is now ready for client connections. This is an informational message; no user action is required.

2020-03-05 09:03:22.55 spid50s Parallel redo is started for database 'Northwind' with worker pool size [2].

2020-03-05 09:03:22.63 spid10s Recovery is writing a checkpoint in database 'msdb' (4). This is an informational message only. No user action is required.

2020-03-05 09:03:22.81 spid10s Parallel redo is shutdown for database 'Northwind' with worker pool size [2].

If there are warnings above and the containers are running - check the logs

In [ ]:
docker logs dbatools_SQL2019_1

Now you can use these containers to run the rest of the notebooks - Don't forget to run the Clean Up Containers Notebook at the end to clean up